In [0]:

--Calculate daily price change, flag direction of trades, and identify streaks of consecutive down days  
with daily_prices as (
  select 
    ticker,
    trade_date,
    close,
    lag(close) over (partition by ticker order by trade_date) as prev_close
    from analytics.stock_prices
),

daily_changes as (
  select*,
 round((close - prev_close)/ prev_close *100,2) as price_change
  from daily_prices
  where prev_close is not null
),

direction_flag as (
  select *,
  case
    when price_change > 0 then 'UP'
    when price_change < 0 then 'DOWN'
    else 'FLAT'
  end as direction
  from daily_changes
),

numbered as (
  select *,
  row_number() over (partition by ticker order by trade_date) as rn1,
  row_number() over (partition by ticker, direction order by trade_date) as rn2
  from direction_flag
),

grouped as (
  select *,
  rn1 - rn2 as streak_group
  from numbered
),

streak_counts as (
  select
    ticker,
    streak_group,
    count(*) as streak_length,
    min(trade_date) as start_date,
    max(trade_date) as end_date
  from grouped
  where direction = 'DOWN'
  group by ticker, streak_group

)

select * 
from streak_counts
where streak_length >= 5
order by ticker, start_date;

--Longest streak of absolute returns > 2%

with daily_prices as (
  select 
    ticker,
    trade_date,
    close,
    lag(close) over (partition by ticker order by trade_date) as prev_close
    from analytics.stock_prices
),

base as (
  select *,
  (close - prev_close) / prev_close as pct_change
  from daily_prices
  where prev_close is not null
),

flagged as (
  select *,
    case
      when abs(pct_change) > 0.02 then 1
      else 0
    end as big_move_flag
  from base
),

numbered as (
  select *,
    row_number() over (
      partition by ticker
      order by trade_date
    ) as rn_all,
    row_number() over (
      partition by ticker, big_move_flag
      order by trade_date
    ) as rn_flag
  from flagged
),

grouped as (
  select *,
  rn_all - rn_flag as streak_group
  from numbered
),

streaks as (
  select
    ticker,
    streak_group,
    count(*) as streak_length,
    min(trade_date) as start_date,
    max(trade_date) as end_date
  from grouped
  where big_move_flag = 1
  group by ticker, streak_group
)

select *
from streaks
where streak_length >= 4
order by ticker, start_date;
